    # to be deleted 

In [ ]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra

In [ ]:
using Flux
using Flux: logitcrossentropy, normalise, onecold, onehotbatch, crossentropy, throttle
using Parameters: @with_kw
using Flux: @epochs

In [ ]:
cv_data = load("../data/experiment/m1/conv_matrix-1.jld2")

cv_matrix = cv_data["cam_1"];

cv_matrix = cv_matrix[1:200, 1:300];

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,4))

ax.pcolormesh(log.(cv_matrix))

ax.set_xlabel("Predicted Signal")
ax.set_ylabel("Observed Signal")

In [ ]:
x_data = []
y_data = []

for ind in CartesianIndices(cv_matrix)
    push!(x_data, [ind.I[1], ind.I[2]])
    append!(y_data, cv_matrix[ind])
end

x_data = hcat(x_data...)
y_data = y_data'

In [ ]:
train_data = Flux.Data.DataLoader(x_data, y_data, batchsize=256, shuffle=true)

In [ ]:
accuracy(x, y, model) = sum((model(x) .- y).^2)
loss(x, y) = crossentropy(model(x), y)  #sum((model(x) .- y).^2)

In [ ]:
model = Chain(Flux.Dense(2, 5, sigmoid), Flux.Dense(5, 1), softmax)

In [ ]:
loss_vector_train = Float64[]

evalcb = () -> begin 
    los_train = loss(x_data, y_data)
    push!(loss_vector_train, los_train)
    @show los_train
end

In [ ]:
# optimiser = Descent(0.5)
optimiser = ADAM()

In [ ]:
@time @epochs 200 Flux.train!(loss, Flux.params(model), train_data, optimiser, cb = throttle(evalcb, 80))